### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk
- 以句點為分隔拆句子

In [2]:
import json
import nltk

In [3]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [8]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-SYM"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-SYM"
        elif i == (ne_idx + ne_len - 1):
            return "E-SYM"
            
    return "O"

In [9]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    sentences = case_content.split("。")
    
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    t += len(main_list)
        
        
    for sent in sentences:
        ne_index_list = []
        for ne in main_list:
            ne_idx = sent.find(ne)
            if ne_idx != -1 :
                ne_index_list.append((ne_idx,len(ne)))

        texts = []
        for i,w in enumerate(sent):
            label = getLabelWithWordIndex(i,ne_index_list)
            texts.append((w,label))

        docs.append(texts)

print(len(docs))

48901


In [10]:
for w in docs[1]:
    print(w[0],w[1])

每 O
于 O
夏 O
日 O
洗 O
澡 O
后 O
即 O
寒 B-SYM
热 I-SYM
不 I-SYM
适 E-SYM
， O
自 O
感 O
发 O
热 O
而 O
又 O
需 O
披 O
衣 O
裹 O
护 O


In [6]:
data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    tagged = nltk.pos_tag(tokens)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

In [6]:
print(len(data))

48901


In [7]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]
        
    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [8]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
print(len(X_train))
print(len(X_test))

39120
9781


In [9]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 30140
Seconds required: 3.140

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 298246.082437
Feature norm: 1.000000
Error norm: 81370.069603
Active features: 30011
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 1.077

***** Iteration #2 *****
Loss: 296436.567042
Feature norm: 1.003453
Error norm: 78831.806424
Active features: 28477
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.547

***** Iteration #3 *****
Loss: 261728.980614
Feature norm: 1.279092
Error norm: 115519.281400
Active features: 20680
Line search trials: 1
Line search step: 1.000000
Seconds required

***** Iteration #41 *****
Loss: 47254.979862
Feature norm: 31.812947
Error norm: 4813.585165
Active features: 25432
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.547

***** Iteration #42 *****
Loss: 47194.863314
Feature norm: 32.589112
Error norm: 65758.053043
Active features: 24679
Line search trials: 3
Line search step: 0.250000
Seconds required for this iteration: 1.639

***** Iteration #43 *****
Loss: 46270.221895
Feature norm: 34.027414
Error norm: 75625.572753
Active features: 24685
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.551

***** Iteration #44 *****
Loss: 44775.243070
Feature norm: 34.709640
Error norm: 20371.906996
Active features: 24834
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.547

***** Iteration #45 *****
Loss: 44394.591206
Feature norm: 35.085827
Error norm: 17237.108842
Active features: 24938
Line search trials: 1
Line search step: 1.00

***** Iteration #81 *****
Loss: 34389.688981
Feature norm: 88.601519
Error norm: 4582.572971
Active features: 22031
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.551

***** Iteration #82 *****
Loss: 34336.732360
Feature norm: 89.886798
Error norm: 7766.490045
Active features: 21970
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.546

***** Iteration #83 *****
Loss: 34287.217009
Feature norm: 90.672347
Error norm: 5242.885517
Active features: 21928
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.550

***** Iteration #84 *****
Loss: 34243.319049
Feature norm: 91.661438
Error norm: 5427.099962
Active features: 21925
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.546

***** Iteration #85 *****
Loss: 34199.878464
Feature norm: 92.194207
Error norm: 3379.520125
Active features: 21833
Line search trials: 1
Line search step: 1.000000

***** Iteration #124 *****
Loss: 33445.249710
Feature norm: 111.593437
Error norm: 4940.824848
Active features: 21154
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.548

***** Iteration #125 *****
Loss: 33435.626398
Feature norm: 111.827466
Error norm: 3571.501781
Active features: 21162
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.547

***** Iteration #126 *****
Loss: 33428.988474
Feature norm: 112.185801
Error norm: 4195.897676
Active features: 21141
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.548

***** Iteration #127 *****
Loss: 33420.830420
Feature norm: 112.396124
Error norm: 3018.151779
Active features: 21121
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.549

***** Iteration #128 *****
Loss: 33414.722853
Feature norm: 112.724979
Error norm: 3948.069067
Active features: 21108
Line search trials: 1
Line search step

***** Iteration #169 *****
Loss: 33180.782382
Feature norm: 121.334906
Error norm: 2088.130155
Active features: 20847
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.549

***** Iteration #170 *****
Loss: 33177.632906
Feature norm: 121.524691
Error norm: 3806.963430
Active features: 20835
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.549

***** Iteration #171 *****
Loss: 33170.953147
Feature norm: 121.652359
Error norm: 2411.695201
Active features: 20840
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.546

***** Iteration #172 *****
Loss: 33168.522454
Feature norm: 121.858913
Error norm: 4219.497632
Active features: 20836
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.544

***** Iteration #173 *****
Loss: 33161.110678
Feature norm: 121.995119
Error norm: 2447.654659
Active features: 20843
Line search trials: 1
Line search step

In [21]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
#i = 12
#for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
#    print("%s (%s)" % (y, x))

for i,doc in enumerate(X_test):
    print(i,'---------')
    for j,word in enumerate(doc):
        w = word[1].split("=")[1]
        y1 = y_pred[i][j]
        y2 = y_test[i][j]
        if '-S' in y1 and y1 != y2:
            print(w,y1,y2)
    

#print(X_test[12][4][1].split("=")[1])
#print(y_pred[12][4])
#print(y_test[12][4])

0 ---------
1 ---------
2 ---------
3 ---------
4 ---------
5 ---------
6 ---------
7 ---------
8 ---------
9 ---------
经 B-S O
前 I-S O
头 I-S O
闷 E-S O
胸 B-S O
闷 E-S O
10 ---------
11 ---------
12 ---------
13 ---------
14 ---------
15 ---------
16 ---------
17 ---------
18 ---------
19 ---------
20 ---------
21 ---------
22 ---------
23 ---------
24 ---------
25 ---------
26 ---------
27 ---------
28 ---------
29 ---------
喉 B-S O
头 I-S O
水 I-S O
肿 E-S O
30 ---------
31 ---------
32 ---------
33 ---------
34 ---------
35 ---------
36 ---------
37 ---------
38 ---------
39 ---------
40 ---------
41 ---------
42 ---------
43 ---------
44 ---------
45 ---------
46 ---------
47 ---------
手 B-S O
麻 E-S O
乏 B-S O
力 E-S O
48 ---------
49 ---------
50 ---------
51 ---------
乏 B-S O
力 E-S O
52 ---------
53 ---------
54 ---------
55 ---------
56 ---------
57 ---------
胸 B-S O
闷 E-S O
胃 B-S O
胀 I-S O
泛 I-S O
酸 E-S O
腰 B-S O
部 I-S O
酸 I-S O
麻 E-S O
四 B-S O
肢 I-S O
关 I-S O
节 I-S O
疼 E-S O
58 -----

749 ---------
750 ---------
751 ---------
752 ---------
753 ---------
754 ---------
755 ---------
756 ---------
757 ---------
758 ---------
759 ---------
760 ---------
761 ---------
头 B-S I-S
762 ---------
763 ---------
咳 B-S O
嗽 E-S O
气 B-S O
短 E-S O
764 ---------
765 ---------
766 ---------
767 ---------
768 ---------
769 ---------
770 ---------
771 ---------
772 ---------
773 ---------
774 ---------
面 B-S O
部 I-S O
抽 I-S B-S
不 B-S O
时 I-S O
眨 I-S O
眼 E-S O
挤 B-S O
眉 I-S O
弄 I-S O
眼 E-S O
上 B-S O
课 I-S O
注 I-S O
意 I-S O
力 I-S O
不 I-S O
集 I-S O
中 E-S O
775 ---------
776 ---------
777 ---------
778 ---------
烦 B-S O
躁 I-S O
谵 I-S O
语 E-S O
779 ---------
780 ---------
781 ---------
782 ---------
783 ---------
784 ---------
785 ---------
晕 E-S I-S
恶 B-S I-S
786 ---------
787 ---------
788 ---------
789 ---------
790 ---------
791 ---------
792 ---------
793 ---------
794 ---------
795 ---------
796 ---------
797 ---------
798 ---------
799 ---------
800 ---------
801 ---------
802 ------

溏 E-S O
疲 B-S O
惫 I-S O
不 I-S O
支 I-S O
而 I-S O
全 E-S O
1322 ---------
1323 ---------
1324 ---------
气 B-S O
短 E-S O
1325 ---------
1326 ---------
1327 ---------
1328 ---------
1329 ---------
1330 ---------
1331 ---------
1332 ---------
1333 ---------
1334 ---------
1335 ---------
1336 ---------
1337 ---------
1338 ---------
1339 ---------
1340 ---------
1341 ---------
1342 ---------
1343 ---------
1344 ---------
1345 ---------
1346 ---------
1347 ---------
1348 ---------
1349 ---------
1350 ---------
1351 ---------
1352 ---------
1353 ---------
1354 ---------
1355 ---------
1356 ---------
1357 ---------
1358 ---------
1359 ---------
1360 ---------
1361 ---------
1362 ---------
1363 ---------
1364 ---------
1365 ---------
心 B-S O
悸 E-S O
1366 ---------
1367 ---------
1368 ---------
1369 ---------
1370 ---------
1371 ---------
1372 ---------
1373 ---------
1374 ---------
心 B-S O
下 I-S O
微 I-S O
作 I-S O
冲 I-S O
痛 E-S O
大 B-S O
便 I-S O
涩 E-S O
1375 ---------
1376 ---------
1377 ---------


1939 ---------
1940 ---------
1941 ---------
1942 ---------
1943 ---------
1944 ---------
眼 B-S O
珠 I-S O
发 I-S O
胀 E-S O
视 B-S O
力 I-S O
昏 I-S O
糊 E-S O
1945 ---------
1946 ---------
1947 ---------
胸 B-S O
闷 I-S O
嗳 I-S O
气 E-S O
1948 ---------
1949 ---------
1950 ---------
1951 ---------
1952 ---------
1953 ---------
1954 ---------
1955 ---------
1956 ---------
差 E-S B-S
尿 B-S O
痛 E-S O
1957 ---------
1958 ---------
口 B-S O
干 E-S O
大 B-S O
便 I-S O
偏 I-S O
干 E-S O
排 B-S O
便 I-S O
难 E-S O
1959 ---------
1960 ---------
1961 ---------
1962 ---------
1963 ---------
1964 ---------
1965 ---------
1966 ---------
1967 ---------
1968 ---------
1969 ---------
1970 ---------
呃 B-S O
逆 E-S O
1971 ---------
1972 ---------
1973 ---------
失 B-S O
眠 E-S O
1974 ---------
1975 ---------
1976 ---------
口 B-S O
能 I-S O
稍 I-S O
张 E-S O
1977 ---------
1978 ---------
1979 ---------
1980 ---------
1981 ---------
1982 ---------
痰 B-S O
鸣 I-S O
量 I-S O
多 E-S O
1983 ---------
1984 ---------
1985 ---------
1986 

耳 B-S O
鸣 E-S O
头 B-S O
晕 E-S O
2550 ---------
2551 ---------
2552 ---------
2553 ---------
2554 ---------
2555 ---------
2556 ---------
面 B-S O
目 I-S O
及 I-S O
双 I-S O
下 I-S O
肢 I-S O
重 I-S O
度 I-S O
浮 I-S O
肿 E-S O
2557 ---------
2558 ---------
2559 ---------
2560 ---------
2561 ---------
2562 ---------
2563 ---------
2564 ---------
2565 ---------
2566 ---------
2567 ---------
2568 ---------
2569 ---------
2570 ---------
2571 ---------
2572 ---------
2573 ---------
2574 ---------
2575 ---------
2576 ---------
2577 ---------
2578 ---------
2579 ---------
2580 ---------
2581 ---------
2582 ---------
2583 ---------
2584 ---------
2585 ---------
2586 ---------
2587 ---------
2588 ---------
2589 ---------
2590 ---------
2591 ---------
2592 ---------
2593 ---------
2594 ---------
2595 ---------
2596 ---------
2597 ---------
2598 ---------
2599 ---------
2600 ---------
咳 B-S O
嗽 E-S O
气 B-S O
短 E-S O
喘 B-S O
憋 E-S O
喉 B-S O
中 I-S O
痰 I-S O
鸣 E-S O
沫 I-S E-S
样 E-S O
鼻 B-S O
塞 E-S O
2601 ----

...

6764 ---------
6765 ---------
6766 ---------
6767 ---------
荨 B-S O
麻 I-S O
疹 E-S O
6768 ---------
6769 ---------
6770 ---------
6771 ---------
6772 ---------
6773 ---------
6774 ---------
6775 ---------
6776 ---------
6777 ---------
6778 ---------
6779 ---------
6780 ---------
6781 ---------
6782 ---------
6783 ---------
6784 ---------
6785 ---------
6786 ---------
头 B-S O
晕 E-S O
6787 ---------
精 B-S O
神 I-S O
萎 I-S O
靡 E-S O
烦 B-S O
躁 E-S O
6788 ---------
6789 ---------
6790 ---------
6791 ---------
头 B-S O
晕 E-S O
食 B-S O
少 I-S O
便 I-S B-S
6792 ---------
6793 ---------
6794 ---------
6795 ---------
6796 ---------
6797 ---------
6798 ---------
6799 ---------
6800 ---------
6801 ---------
头 B-S O
胀 E-S O
6802 ---------
6803 ---------
头 B-S O
晕 E-S O
失 B-S O
眠 E-S O
6804 ---------
6805 ---------
6806 ---------
6807 ---------
6808 ---------
失 B-S O
眠 E-S O
时 B-S O
烘 I-S B-S
6809 ---------
6810 ---------
6811 ---------
6812 ---------
热 I-S E-S
出 I-S B-S
6813 ---------
6814 ---------
681

7369 ---------
7370 ---------
7371 ---------
7372 ---------
7373 ---------
7374 ---------
7375 ---------
嗳 B-S O
气 E-S O
7376 ---------
7377 ---------
7378 ---------
7379 ---------
7380 ---------
7381 ---------
头 B-S O
晕 E-S O
双 B-S O
下 I-S O
肢 I-S O
乏 I-S O
力 E-S O
7382 ---------
7383 ---------
7384 ---------
7385 ---------
7386 ---------
7387 ---------
7388 ---------
7389 ---------
7390 ---------
7391 ---------
7392 ---------
7393 ---------
7394 ---------
7395 ---------
7396 ---------
7397 ---------
7398 ---------
7399 ---------
失 B-S O
眠 E-S O
7400 ---------
7401 ---------
7402 ---------
7403 ---------
7404 ---------
7405 ---------
7406 ---------
7407 ---------
7408 ---------
7409 ---------
7410 ---------
7411 ---------
肢 B-S O
体 I-S O
麻 I-S O
木 I-S O
而 I-S O
兼 I-S O
抽 I-S O
搐 E-S O
7412 ---------
7413 ---------
胸 B-S O
闷 E-S O
7414 ---------
胸 B-S O
闷 I-S O
气 I-S O
急 E-S O
畏 B-S O
寒 I-S O
怕 I-S O
冷 E-S O
7415 ---------
7416 ---------
7417 ---------
7418 ---------
7419 ---------
742

目 I-S O
不 I-S O
语 E-S O
7959 ---------
烦 B-S O
躁 I-S O
易 I-S O
怒 E-S O
失 B-S O
眠 I-S O
多 I-S O
梦 E-S O
7960 ---------
7961 ---------
7962 ---------
7963 ---------
7964 ---------
7965 ---------
7966 ---------
7967 ---------
咳 B-S O
喘 E-S O
7968 ---------
7969 ---------
7970 ---------
7971 ---------
7972 ---------
7973 ---------
7974 ---------
7975 ---------
7976 ---------
7977 ---------
7978 ---------
咳 B-S O
嗽 E-S O
7979 ---------
7980 ---------
7981 ---------
7982 ---------
7983 ---------
7984 ---------
7985 ---------
7986 ---------
7987 ---------
7988 ---------
7989 ---------
7990 ---------
7991 ---------
7992 ---------
7993 ---------
7994 ---------
7995 ---------
7996 ---------
7997 ---------
7998 ---------
7999 ---------
8000 ---------
8001 ---------
8002 ---------
8003 ---------
8004 ---------
8005 ---------
8006 ---------
8007 ---------
便 B-S O
溏 E-S O
尿 B-S O
少 E-S O
面 B-S O
色 I-S O
无 I-S O
华 E-S O
8008 ---------
8009 ---------
8010 ---------
8011 ---------
无 B-S O
汗 E-S O
胸 B-S

肢 I-S O
冷 E-S O
恶 B-S O
寒 E-S O
8631 ---------
8632 ---------
8633 ---------
8634 ---------
8635 ---------
8636 ---------
8637 ---------
心 B-S O
悸 E-S O
8638 ---------
8639 ---------
8640 ---------
8641 ---------
8642 ---------
8643 ---------
8644 ---------
8645 ---------
8646 ---------
8647 ---------
行 B-S O
走 I-S O
欲 I-S O
跑 I-S O
状 E-S O
8648 ---------
8649 ---------
8650 ---------
8651 ---------
8652 ---------
8653 ---------
8654 ---------
8655 ---------
8656 ---------
8657 ---------
8658 ---------
8659 ---------
8660 ---------
8661 ---------
8662 ---------
8663 ---------
眠 I-S E-S
盗 I-S B-S
8664 ---------
8665 ---------
8666 ---------
8667 ---------
8668 ---------
8669 ---------
口 B-S O
干 E-S O
8670 ---------
8671 ---------
8672 ---------
8673 ---------
8674 ---------
8675 ---------
8676 ---------
8677 ---------
8678 ---------
8679 ---------
8680 ---------
8681 ---------
8682 ---------
8683 ---------
8684 ---------
8685 ---------
8686 ---------
8687 ---------
8688 ---------
胸 B-S 

疲 I-S O
劳 E-S O
面 B-S O
部 I-S O
起 I-S O
红 I-S O
色 I-S O
小 I-S O
丘 I-S O
疹 E-S O
9244 ---------
9245 ---------
9246 ---------
9247 ---------
9248 ---------
9249 ---------
9250 ---------
9251 ---------
9252 ---------
9253 ---------
9254 ---------
9255 ---------
左 B-S O
侧 I-S O
偏 I-S B-S
9256 ---------
9257 ---------
9258 ---------
9259 ---------
9260 ---------
9261 ---------
9262 ---------
9263 ---------
9264 ---------
9265 ---------
9266 ---------
9267 ---------
9268 ---------
9269 ---------
下 B-S O
肢 I-S O
无 I-S O
汗 E-S O
9270 ---------
9271 ---------
9272 ---------
9273 ---------
9274 ---------
9275 ---------
9276 ---------
9277 ---------
9278 ---------
9279 ---------
9280 ---------
9281 ---------
9282 ---------
9283 ---------
头 B-S O
昏 E-S O
9284 ---------
9285 ---------
9286 ---------
9287 ---------
恶 B-S O
心 E-S O
9288 ---------
9289 ---------
9290 ---------
9291 ---------
9292 ---------
9293 ---------
9294 ---------
9295 ---------
9296 ---------
9297 ---------
9298 ---------
9299 

In [29]:
for i,doc in enumerate(X_test):
    print(i,'---------')
    for j,word in enumerate(doc):
        w = word[1].split("=")[1]
        y1 = y_pred[i][j]
        y2 = y_test[i][j]
        if '-S' in y2 and y1 != y2:
            print(w,y1,y2)

0 ---------
1 ---------
2 ---------
3 ---------
4 ---------
5 ---------
6 ---------
7 ---------
8 ---------
9 ---------
目 O B-S
赤 O E-S
10 ---------
11 ---------
头 O B-S
晕 O E-S
胸 O B-S
闷 O E-S
胸 O B-S
痛 O E-S
乏 O B-S
力 O E-S
神 O B-S
疲 O E-S
12 ---------
13 ---------
14 ---------
15 ---------
16 ---------
17 ---------
18 ---------
19 ---------
腰 O B-S
酸 O E-S
20 ---------
21 ---------
22 ---------
23 ---------
24 ---------
25 ---------
26 ---------
27 ---------
28 ---------
29 ---------
30 ---------
31 ---------
32 ---------
33 ---------
34 ---------
35 ---------
36 ---------
37 ---------
38 ---------
39 ---------
40 ---------
41 ---------
42 ---------
43 ---------
44 ---------
45 ---------
46 ---------
47 ---------
48 ---------
49 ---------
失 O B-S
眠 O E-S
50 ---------
51 ---------
52 ---------
53 ---------
54 ---------
55 ---------
56 ---------
57 ---------
58 ---------
59 ---------
心 O B-S
烦 O I-S
急 O I-S
躁 O E-S
60 ---------
61 ---------
62 ---------
63 ---------
64 ---------
65 --

着 O I-S
棉 O I-S
裤 O I-S
亦 O I-S
无 O I-S
热 O I-S
感 O E-S
上 O B-S
半 O I-S
身 O I-S
时 O I-S
热 O E-S
继 O B-S
之 O I-S
畏 O I-S
寒 O E-S
511 ---------
512 ---------
513 ---------
514 ---------
515 ---------
516 ---------
低 O B-S
烧 O E-S
517 ---------
518 ---------
519 ---------
520 ---------
521 ---------
522 ---------
523 ---------
524 ---------
525 ---------
526 ---------
527 ---------
528 ---------
529 ---------
530 ---------
531 ---------
532 ---------
533 ---------
534 ---------
535 ---------
神 O B-S
疲 O E-S
烦 O B-S
躁 O E-S
536 ---------
537 ---------
538 ---------
539 ---------
540 ---------
541 ---------
542 ---------
543 ---------
544 ---------
545 ---------
546 ---------
547 ---------
量 O B-S
少 O E-S
548 ---------
549 ---------
550 ---------
551 ---------
552 ---------
553 ---------
554 ---------
555 ---------
556 ---------
557 ---------
558 ---------
559 ---------
大 O B-S
便 O I-S
溏 O E-S
便 O B-S
后 O I-S
失 O I-S
禁 O E-S
右 O B-S
胁 O I-S
隐 O I-S
痛 O E-S
560 ---------
561 ---------
562 --

不 O B-S
易 O I-S
咯 O I-S
出 O E-S
877 ---------
878 ---------
879 ---------
880 ---------
881 ---------
882 ---------
883 ---------
884 ---------
885 ---------
886 ---------
887 ---------
888 ---------
889 ---------
890 ---------
891 ---------
形 O B-S
寒 O E-S
闷 I-S E-S
便 I-S B-S
溏 I-S E-S
892 ---------
893 ---------
发 O B-S
热 O E-S
894 ---------
895 ---------
896 ---------
897 ---------
898 ---------
899 ---------
900 ---------
901 ---------
902 ---------
903 ---------
904 ---------
905 ---------
906 ---------
907 ---------
908 ---------
909 ---------
910 ---------
911 ---------
912 ---------
皮 O B-S
肤 O I-S
反 O I-S
复 O I-S
溃 O I-S
烂 O E-S
913 ---------
914 ---------
915 ---------
916 ---------
917 ---------
918 ---------
919 ---------
920 ---------
921 ---------
922 ---------
923 ---------
924 ---------
925 ---------
926 ---------
927 ---------
928 ---------
929 ---------
930 ---------
931 ---------
932 ---------
933 ---------
934 ---------
935 ---------
936 ---------
937 ---------
938 

红 O E-S
经 O B-S
前 O I-S
乳 O I-S
胀 O E-S
1449 ---------
1450 ---------
1451 ---------
1452 ---------
1453 ---------
1454 ---------
1455 ---------
1456 ---------
1457 ---------
1458 ---------
1459 ---------
1460 ---------
1461 ---------
1462 ---------
1463 ---------
1464 ---------
1465 ---------
1466 ---------
1467 ---------
1468 ---------
1469 ---------
1470 ---------
1471 ---------
1472 ---------
1473 ---------
1474 ---------
1475 ---------
1476 ---------
1477 ---------
1478 ---------
1479 ---------
1480 ---------
1481 ---------
1482 ---------
1483 ---------
1484 ---------
1485 ---------
1486 ---------
1487 ---------
1488 ---------
1489 ---------
1490 ---------
1491 ---------
1492 ---------
失 O B-S
眠 O E-S
1493 ---------
1494 ---------
麻 O B-S
木 O E-S
便 O B-S
秘 O E-S
1495 ---------
心 O B-S
下 O I-S
痛 O E-S
1496 ---------
1497 ---------
1498 ---------
1499 ---------
经 O B-S
行 O I-S
量 O I-S
多 O I-S
如 O I-S
崩 O E-S
1500 ---------
1501 ---------
1502 ---------
胸 O B-S
痛 O E-S
1503 ---------

口 O B-S
干 O I-S
黏 O I-S
腻 O E-S
1996 ---------
1997 ---------
1998 ---------
头 O B-S
面 O I-S
红 O I-S
肿 O E-S
1999 ---------
2000 ---------
2001 ---------
2002 ---------
2003 ---------
2004 ---------
2005 ---------
2006 ---------
心 O B-S
烦 O I-S
胸 O I-S
满 O E-S
2007 ---------
2008 ---------
2009 ---------
2010 ---------
2011 ---------
2012 ---------
2013 ---------
2014 ---------
2015 ---------
2016 ---------
2017 ---------
2018 ---------
2019 ---------
2020 ---------
咽 O B-S
干 O E-S
2021 ---------
2022 ---------
2023 ---------
心 O B-S
慌 O E-S
胃 O B-S
脘 O I-S
部 O I-S
不 O I-S
适 O E-S
饱 O B-S
胀 O I-S
感 O E-S
食 O B-S
后 O I-S
呃 B-S I-S
2024 ---------
2025 ---------
白 O B-S
带 O I-S
量 O I-S
多 O E-S
色 O B-S
白 O E-S
无 O B-S
异 O I-S
味 O E-S
2026 ---------
2027 ---------
2028 ---------
2029 ---------
2030 ---------
2031 ---------
2032 ---------
2033 ---------
2034 ---------
2035 ---------
2036 ---------
2037 ---------
2038 ---------
2039 ---------
2040 ---------
2041 ---------
2042 ---------
2043 

...

热 O I-S
往 O I-S
来 O E-S
7704 ---------
7705 ---------
7706 ---------
7707 ---------
7708 ---------
7709 ---------
7710 ---------
7711 ---------
7712 ---------
7713 ---------
7714 ---------
7715 ---------
7716 ---------
乳 O B-S
胀 O E-S
7717 ---------
7718 ---------
7719 ---------
7720 ---------
7721 ---------
7722 ---------
7723 ---------
言 O B-S
语 O I-S
不 O I-S
利 O E-S
纳 O B-S
食 O I-S
不 O I-S
佳 O E-S
7724 ---------
7725 ---------
7726 ---------
7727 ---------
7728 ---------
7729 ---------
7730 ---------
7731 ---------
双 O B-S
下 O I-S
肢 O I-S
浮 O I-S
肿 O E-S
小 O B-S
便 O I-S
量 O I-S
少 O E-S
7732 ---------
7733 ---------
7734 ---------
7735 ---------
7736 ---------
7737 ---------
7738 ---------
7739 ---------
7740 ---------
7741 ---------
7742 ---------
7743 ---------
7744 ---------
7745 ---------
呕 O B-S
吐 O E-S
7746 ---------
7747 ---------
7748 ---------
7749 ---------
7750 ---------
7751 ---------
头 O B-S
晕 O E-S
7752 ---------
7753 ---------
下 O B-S
腹 O I-S
疼 O I-S
痛 O E-S
带 O B-S
下 

8203 ---------
8204 ---------
8205 ---------
8206 ---------
8207 ---------
8208 ---------
8209 ---------
8210 ---------
8211 ---------
8212 ---------
8213 ---------
8214 ---------
8215 ---------
8216 ---------
8217 ---------
8218 ---------
8219 ---------
8220 ---------
8221 ---------
8222 ---------
8223 ---------
8224 ---------
8225 ---------
8226 ---------
8227 ---------
8228 ---------
心 O B-S
烦 O I-S
梦 O I-S
多 O E-S
8229 ---------
8230 ---------
8231 ---------
8232 ---------
8233 ---------
8234 ---------
8235 ---------
8236 ---------
8237 ---------
皮 O B-S
肤 O I-S
瘙 O I-S
痒 O E-S
8238 ---------
8239 ---------
神 I-S B-S
尿 I-S B-S
8240 ---------
8241 ---------
8242 ---------
8243 ---------
8244 ---------
8245 ---------
8246 ---------
8247 ---------
8248 ---------
8249 ---------
8250 ---------
8251 ---------
夜 O B-S
寐 O I-S
早 O I-S
醒 O E-S
醒 O B-S
后 O I-S
浅 O I-S
寐 O E-S
8252 ---------
8253 ---------
8254 ---------
8255 ---------
8256 ---------
腰 O B-S
腿 O I-S
酸 O I-S
软 O E-S
8257 -----

腰 O B-S
部 O I-S
酸 O I-S
楚 O E-S
小 O B-S
腹 O I-S
胀 O E-S
8667 ---------
8668 ---------
8669 ---------
8670 ---------
8671 ---------
8672 ---------
8673 ---------
8674 ---------
8675 ---------
8676 ---------
8677 ---------
腹 O B-S
中 O I-S
饥 O I-S
饿 O I-S
感 O E-S
大 O B-S
便 O I-S
不 O I-S
畅 O E-S
8678 ---------
8679 ---------
8680 ---------
8681 ---------
8682 ---------
8683 ---------
8684 ---------
8685 ---------
8686 ---------
8687 ---------
8688 ---------
8689 ---------
干 O B-S
咳 O E-S
8690 ---------
8691 ---------
8692 ---------
8693 ---------
8694 ---------
胸 O B-S
闷 O E-S
气 O B-S
短 O E-S
8695 ---------
8696 ---------
8697 ---------
8698 ---------
8699 ---------
8700 ---------
8701 ---------
尿 O B-S
不 O I-S
净 O E-S
尿 O B-S
血 O E-S
8702 ---------
哮 O B-S
喘 O E-S
8703 ---------
8704 ---------
8705 ---------
8706 ---------
8707 ---------
8708 ---------
8709 ---------
8710 ---------
8711 ---------
8712 ---------
8713 ---------
8714 ---------
胃 O B-S
脘 O I-S
部 O I-S
隐 O I-S
痛 O E-S
胀 O B-S


9092 ---------
9093 ---------
9094 ---------
9095 ---------
9096 ---------
9097 ---------
9098 ---------
9099 ---------
9100 ---------
9101 ---------
9102 ---------
自 O B-S
汗 O E-S
9103 ---------
9104 ---------
9105 ---------
9106 ---------
9107 ---------
9108 ---------
9109 ---------
利 O B-S
十 O I-S
余 O I-S
日 O E-S
呼 O B-S
之 O I-S
不 O I-S
应 O E-S
9110 ---------
9111 ---------
9112 ---------
9113 ---------
9114 ---------
反 O B-S
酸 O E-S
9115 ---------
9116 ---------
9117 ---------
9118 ---------
9119 ---------
9120 ---------
有 O B-S
血 O I-S
块 O E-S
腰 O B-S
酸 O E-S
乳 O B-S
胀 O E-S
冷 O B-S
汗 O I-S
出 O E-S
9121 ---------
9122 ---------
9123 ---------
9124 ---------
9125 ---------
周 O B-S
身 O I-S
浮 O I-S
肿 O E-S
9126 ---------
9127 ---------
9128 ---------
9129 ---------
9130 ---------
9131 ---------
痛 O B-S
经 O E-S
9132 ---------
9133 ---------
9134 ---------
9135 ---------
9136 ---------
9137 ---------
9138 ---------
9139 ---------
口 O B-S
干 O I-S
舌 O I-S
燥 O E-S
尿 O B-S
黄 O E-S
9140 ---

9516 ---------
皮 O B-S
肤 O E-S
9517 ---------
9518 ---------
9519 ---------
9520 ---------
9521 ---------
9522 ---------
半 O B-S
身 O I-S
不 O I-S
遂 O E-S
9523 ---------
9524 ---------
头 O B-S
晕 O I-S
或 O I-S
痛 O E-S
月 O B-S
经 O I-S
量 O I-S
多 O E-S
9525 ---------
肋 O B-S
间 O I-S
神 O I-S
经 O I-S
痛 O E-S
9526 ---------
9527 ---------
9528 ---------
9529 ---------
左 O B-S
右 O I-S
颈 O I-S
部 O I-S
有 O I-S
数 O I-S
个 O I-S
淋 O I-S
巴 O I-S
结 O I-S
肿 O I-S
大 O E-S
左 O B-S
侧 O I-S
的 O I-S
淋 O I-S
巴 O I-S
结 O I-S
可 O I-S
见 O I-S
瘘 O I-S
孔 O E-S
有 O B-S
脓 O I-S
水 O I-S
流 O I-S
出 O E-S
9530 ---------
9531 ---------
9532 ---------
9533 ---------
9534 ---------
9535 ---------
9536 ---------
9537 ---------
9538 ---------
9539 ---------
9540 ---------
9541 ---------
9542 ---------
9543 ---------
9544 ---------
腰 O B-S
痛 O E-S
9545 ---------
9546 ---------
9547 ---------
9548 ---------
9549 ---------
9550 ---------
9551 ---------
9552 ---------
心 O B-S
慌 O E-S
气 O B-S
憋 O E-S
9553 ---------
9554 ---------

In [34]:
s = []
for w in X_test[11]: 
    s.append(w[1].split("=")[1])
print(''.join(s))

三诊(1987年10月23日）：服上药7剂，心慌心悸、头晕胀痛基本消失；胸闷胸痛基本缓解；乏力神疲、失眠烦躁明显改善


In [20]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["O", "B-SYM","I-SYM","E-SYM"]))

             precision    recall  f1-score   support

          O       0.97      0.99      0.98    307289
      B-SYM       0.59      0.46      0.52      4414
      I-SYM       0.52      0.33      0.41      5369
      E-SYM       0.60      0.47      0.53      4396

avg / total       0.96      0.96      0.96    321468

